<a href="https://colab.research.google.com/github/Existanze54/sirius-machine-learning-2024/blob/main/Seminars/S3_Reg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Основные алгоритмы ML

### Семинар 3. Модели регрессии: kNN, линейная, логистическая. Метрики качества RMSE, MAE, коэффициент корреляции r-Пирсона

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

sns.set()
sns.set_style("whitegrid")

### Задача 1: Предсказание растворимости белков

В этой задаче предлагается произвести предсказание растворимости различных белков, используя их физико-химические свойства и белковый состав.

In [ ]:
!wget https://data.bioml.ru/htdocs/courses/bioml/classic_ml/supervised/regression/data/solubility_data.csv

--2024-11-13 04:47:07--  https://data.bioml.ru/htdocs/courses/bioml/classic_ml/supervised/regression/data/solubility_data.csv
Resolving data.bioml.ru (data.bioml.ru)... 188.120.231.217
Connecting to data.bioml.ru (data.bioml.ru)|188.120.231.217|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 184108 (180K) [text/csv]
Saving to: ‘solubility_data.csv’

solubility_data.csv 100%[===================>] 179.79K   496KB/s    in 0.4s    

2024-11-13 04:47:08 (496 KB/s) - ‘solubility_data.csv’ saved [184108/184108]



Датасет `solubility_data.csv` загружен за вас, определены `X` и `y`.

In [ ]:
sol = pd.read_csv("./solubility_data.csv", header=0, index_col=0)
sol.head(3)

solubility  protein_sasa  length  molecular_weight  \
uniprot_id                                                       
P32643              78  14538.357720     293        33936.5675   
Q46829              70  19283.018950     470        54516.1438   
P65556              19   8407.794332     157        17692.5095   

            instability_index  isoelectric_point  helix_fraction  \
uniprot_id                                                         
P32643              40.259795           5.866034        0.310580   
Q46829              34.295149           5.655559        0.329787   
P65556              54.505159           4.679577        0.286624   

            turn_fraction  sheet_fraction  \
uniprot_id                                  
P32643           0.208191        0.187713   
Q46829           0.214894        0.225532   
P65556           0.165605        0.312102   

            reduced_molar_extinction_coefficient  ...   A2   B2    C2  A3  \
uniprot_id                                        ...                       
P32643                                     59360  ...    0  0.0   0.0   0   
Q46829                                    111160  ...  279  1.1  92.5   0   
P65556                                     24980  ...    0  0.0   0.0   0   

             B3   C3  A4   B4  C4  TM  
uniprot_id                             
P32643      0.0  0.0   0  0.0   0   0  
Q46829      0.0  0.0   0  0.0   0   0  
P65556      0.0  0.0   0  0.0   0   0  

[3 rows x 46 columns]

In [ ]:
X = sol.iloc[:,1:]
y = sol["solubility"]

**Разбейте X и y на тренировочную и тестовую выборки**, как в лекции. Соотношение train/test поставьте равным 4:1.

**Осуществите стандартизацию признаков в выборке**.

**Импортируйте модель регрессии k-NN и обучите ее (`model.fit`)**.

**Получите предсказание на оставшейся выборке**. Оцените его качество метриками RMSE, MAE.

**Постройте диаграмму рассеяния** предсказываемого значения в зависимости от истинного.

Следующая функция строит график зависимости качества на тестовой и тренировочной выборках от параметра модели.\
Измените помеченную строку в функции так, чтобы ее можно было использовать **для построения графика зависимости качества от количества соседей для модели k-NN**.

In [ ]:
from sklearn.metrics import mean_squared_error

def make_qual_chart(X_train, X_test, y_train, y_test, metric=mean_squared_error, init=1, limit=50, step=1):
    train_quals = np.zeros(limit)
    test_quals = np.zeros(limit)
    param_range = np.arange(init, limit + 1, step)

    for ind, value in enumerate(param_range):
        ### CHANGE THE NEXT LINE!
        model = None
        # FIT
        model.fit(X_train, y_train)
        # PREDICT
        y_train_pred = model.predict(X_train)
        y_test_pred = model.predict(X_test)
        # EVALUATE QUALITY
        train_quals[ind] = metric(y_true=y_train, y_pred=y_train_pred)
        test_quals[ind] = metric(y_true=y_test, y_pred=y_test_pred)

    # baseline = metric(y_true=y_test, y_pred=np.full_like(y_test, y_train.mean()))

    # TRACING CHART
    plt.figure(figsize=(10, 7))
    plt.plot(param_range, train_quals, label="Train")
    plt.plot(param_range, test_quals, label="Test")
    # plt.axhline(baseline, color="k", ls="--", lw=1.5, label="Baseline")
    plt.legend()
    plt.show()
    #return param_range, train_quals, test_quals

In [ ]:
make_qual_chart(...)

Сделайте вывод о том, **какое число соседей является оптимальным для данной задачи**. Создайте отдельную модель с оптимальным параметром `n_neighbors` и постройте для нее график зависимости предсказываемого значения от истинного.

**Дополнительно**: сравните kNN-regressor с моделью линейной регрессии на тех же данных.

### Задача 2: Обучение на геномных эмбеддингах

В этой задаче предлагается произвести предсказание изменения экспрессии гена в зависимости от разных однонуклеотидных замен в регуляторном участке ДНК, основываясь на нейросетевых геномных эмбеддингах этого участка.

In [ ]:
!wget https://data.bioml.ru/htdocs/courses/bioml/classic_ml/supervised/knn/data/tfs_deepsea.csv

--2024-11-13 04:47:15--  https://data.bioml.ru/htdocs/courses/bioml/classic_ml/supervised/knn/data/tfs_deepsea.csv
Resolving data.bioml.ru (data.bioml.ru)... 188.120.231.217
Connecting to data.bioml.ru (data.bioml.ru)|188.120.231.217|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1098058 (1.0M) [text/csv]
Saving to: ‘tfs_deepsea.csv’

tfs_deepsea.csv     100%[===================>]   1.05M  1.75MB/s    in 0.6s    

2024-11-13 04:47:16 (1.75 MB/s) - ‘tfs_deepsea.csv’ saved [1098058/1098058]



Загрузите датасет `tfs_deepsea.csv`, в качестве `y` используйте колонку `Direction`, в качестве `X` - все колонки, начиная с девятой.

**Разбейте X и y на тренировочную и тестовую выборки**, как в лекции. Соотношение train/test поставьте равным 4:1.\
*Подсказка*: воспользуйтесь функцией `train_test_split`.

**Осуществите стандартизацию признаков в выборке**.

**Импортируйте модель линейной регрессии и обучите ее на соответствующей части выборки (`model.fit`)**.

**Получите предсказание на оставшейся выборке**. Оцените его качество метриками MSE и RMSE. Насколько хороша модель?

Оцените качество графически и с помощью коэффициента корреляции Пирсона.

**Воспользуйтесь моделью kNN-regressor**. Оцените ее качество.

Разбейте X и y на тренировочную и тестовую выборки, **так, как указано в ячейке ниже**. Повторите все шаги, чтобы определить качество регрессии для линейной и kNN-регрессий.

Биологический смысл указанного разбиения: использовать для обучения один промоторный регион при осуществлении предсказания на другом.

In [ ]:
X_train = X[tfs["construction"] == "ECR11"].values
X_test = X[tfs["construction"] == "ALDOB"].values
y_train = y[tfs["construction"] == "ECR11"].values
y_test = y[tfs["construction"] == "ALDOB"].values

**Сделайте выводы о пригодности модели линейной регрессии для такого анализа**. Есть ли здесь биологический смысл?

### Задача 3: Классификация химических веществ

In [ ]:
!wget --no-check-certificate 'https://data.bioml.ru/htdocs/courses/bioml/classic_ml/supervised/knn/data/bace.csv' -O bace.csv

--2022-09-19 20:00:35--  https://data.bioml.ru/htdocs/courses/bioml/classic_ml/supervised/knn/data/bace.csv
Resolving data.bioml.ru (data.bioml.ru)... 178.62.218.34
Connecting to data.bioml.ru (data.bioml.ru)|178.62.218.34|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3897767 (3.7M) [text/csv]
Saving to: ‘bace.csv’

bace.csv            100%[===================>]   3.72M  5.57MB/s    in 0.7s    

2022-09-19 20:00:36 (5.57 MB/s) - ‘bace.csv’ saved [3897767/3897767]



Датасет [BACE](http://moleculenet.ai/datasets-1) загружен за вас, определены `X`, `y` и `smiles`.

In [ ]:
bace = pd.read_csv("./bace.csv", header=0)
bace.head(3)

In [ ]:
X = bace.iloc[:,4:].values
y = bace["Class"].values
smiles = bace["mol"].values

**Разбейте X и y на тренировочную и тестовую выборки**, как в лекции. Соотношение train/test поставьте равным 3:1.\
*Подсказка*: воспользуйтесь функцией `train_test_split`.

**Осуществите стандартизацию признаков в выборке**.

**Импортируйте модель логистической регрессии и обучите ее на соответствующей части выборки (`model.fit`)**.


Для модели `LogisticRegression` укажите параметры `max_iter=200, penalty=None`.

**Получите предсказание на оставшейся выборке**. Оцените его качество метриками из отчета о классификации (`classification_report`).

**Регуляризация ElasticNet** вносит в функцию потерь линейную комбинацию регуляризаций L1 и L2. Подберите оптимальное соотношение для этих данных. Для определения требуемого параметра (соотношение L1 и L2) воспользуйтесь документацией конструктора модели `LogisticRegression`

In [ ]:
from tqdm import tqdm
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression

init = 0
limit = 1
step = 0.05

param_range = np.arange(init, limit + step, step)
train_quals = np.zeros(len(param_range))
test_quals = np.zeros(len(param_range))

# baseline = f1_score(y_true=y_test, y_pred=np.full_like(y_test, y_train.mean()))

for ind, value in tqdm(enumerate(param_range), total=len(param_range)):
    # CREATING MODEL (CHANGE NEXT LINE!)
    model = LogisticRegression(max_iter=10000, solver="saga",
                               penalty="elasticnet")

    # FIT
    model.fit(X_train_scaled, y_train)

    # PREDICT
    y_train_pred = model.predict(X_train_scaled)
    y_test_pred = model.predict(X_test_scaled)

    # EVALUATE QUALITY
    train_quals[ind] = f1_score(y_true=y_train, y_pred=y_train_pred)
    test_quals[ind] = f1_score(y_true=y_test, y_pred=y_test_pred)

In [ ]:
# TRACING CHART
plt.figure(figsize=(10, 7))
plt.plot(param_range, train_quals, label="Train")
plt.plot(param_range, test_quals, label="Test")
# plt.axhline(baseline, color="k", ls="--", lw=1.5, label="Baseline")
plt.legend()
plt.show()

**Сделайте выводы**.